In [2]:
#Code for final report figure, do not run this for processing, display purposes only

from osgeo import gdal
import matplotlib.pyplot as plt
import numpy as np
import rasterio as rio
from rasterio.plot import show
from rasterio.mask import mask
import pandas as pd
import fiona
import os
import glob
from scipy import sparse
%matplotlib inline

In [3]:
#Identify watershed by three letter acronymn and phase
wshed = "VAN"
phase = "P01"
subbasin= "Seymour"

#raster cell resolution
resolution = 1

name = wshed + "_" + str(phase)
subbasin_name = str(wshed + "_" + str(phase) + "_" + subbasin)

#shapefile is the basin used to mask the rasters, should include the total lidar area if not a sub basin
basin = subbasin + ".shp"

#load in SWE multipliers and filter by watershed
SWE_2021 = pd.read_csv(r"H:\ACO Snow Blitz\SWE_multiplier_2021.csv")
SWE_2021 = SWE_2021[(SWE_2021['wshed'] == wshed)]

#File directories for all watersheds
BE_list = {"EGM" : r"H:\Englishman\2021\DEMs\bare_earth\EGM_BE_wgs84_1m.tif", 
      "VAN": r"H:\Metro-Van\2021\DEM\Baseline_Master_Dataset_WGS84\VAN_snowfree_merged_wgs84_utm10_itrf08_epoch2002_dz20cm_1mDEM.tif",
     "CRU" : r"H:\Cruickshank\2021\DEMs\Bare_earth\21_3011_00_snow_free_complete_utmz10_1mDEM.tif",
     "TSI": r"H:\Tsitika\2021\DEM\Bare_Earth\Tsitika_3m_Snow_Free_DTM_WGS84_z9_ellips.tif"}

aspect_list = {"EGM" : r"H:\Englishman\2021\GIS\aspect.tif", "VAN": r"H:\Metro-Van\2021\GIS\aspect.tif"}

snow_depth_list = {"EGM": r"H:\\Englishman\\2021\GIS\\snow_depth_rasters\\" + name + "_SD_clean.tif",
             "VAN": r"H:\\Metro-Van\\2021\GIS\\snow_depth_rasters\\" + name + "_warp.tif"}

basins = {"EGM": ["arrowsmith", "cokely", "fishtail"],"VAN": ["useymour_nolakes"]}
basin_directory = {"EGM": r"H:\Englishman\2021\GIS\boundaries", "VAN" : r"H:\Metro-Van\2021\GIS\wshed_boundaries"}

#swe table output for matrices
swe_table_list = {"EGM": r"H:\Englishman\2021\Analysis\swe_tables", "VAN": r"H:\Metro-Van\2021\Analysis\swe_tables"}
                   
basin = basins[wshed][0]
basin_fp = os.path.join(basin_directory[wshed],basin + ".shp")


In [ ]:
#Calculate the multipliers, run function for low confidence interval, SWE muliplier, and high confidence interval
def SWE_multiply(parameter,phase):
    parameter = SWE_2021.loc[SWE_2021['phase'] == phase, parameter].item()
    return parameter

l_ci= SWE_multiply("l_ci", phase)
h_ci = SWE_multiply("h_ci", phase)
multiplier = SWE_multiply("multiplier", phase)

lm = l_ci/multiplier 
um = h_ci/multiplier

#Checks
print("Phase: " + str(name))
print("low CI: " + str(l_ci), str(lm))
print("high CI: " + str(h_ci), str(um))
print("multiplier: " + str(multiplier))

In [ ]:
#extract the cooridnates of the sub basin shapefile 
def coords (basin_directory):
    with fiona.open(basin_directory) as shapefile:
        print(shapefile)
        for feature in shapefile:
            shapes = [feature["geometry"]]
    return shapes

In [ ]:
#call function - output should be paired coordinates
mask_coords = coords(basin_fp)
display(mask_coords)
type(mask_coords)

In [ ]:
#call function to mask the 
def open_masked_raster_array(tif):
    with rio.open(tif) as src:
        raster_image, raster_transform = rio.mask.mask(src, mask_coords, crop=True)
        return raster_image

In [ ]:
%%time
#mask the bare earth, check it displays
be_dem = open_masked_raster_array(BE_list[wshed])
be_dem = be_dem[0,:,:]
aspect = open_masked_raster_array(aspect_list[wshed])
aspect = aspect[0,:,:]
snow_depth = open_masked_raster_array(snow_depth_list[wshed])
snow_depth = snow_depth[0,:,:]

In [ ]:
aspect[be_dem < 0] = np.nan
snow_depth[be_dem < 0]= np.nan
be_dem[be_dem < 0] = np.nan
#unravel the rasters
be_dem_ravel = np.ravel(be_dem)
aspect_ravel = np.ravel(aspect)
snow_depth_ravel = np.ravel(snow_depth)
#Convert the unravelled arrays into a data frame form
df = pd.DataFrame({"Elev": be_dem_ravel, "Asp": aspect_ravel, "SD": snow_depth_ravel})
#Change the no data values of snow depth and aspect to NP.nodata, then delete from DF
df['Asp'] = df['Asp'].map(lambda x: np.nan if x <-1  else x)
df['SD'] = df['SD'].map(lambda x: np.nan if x < -0.2  else x)
df = df.dropna()
df = df.reset_index(drop=True)
df['zbin'] = round(df["Elev"], -2)
df["asp"]= round(df["Asp"])

In [ ]:
#calc
def aspect(df, old_col, new_col):
    conditions_asp = [
        df[old_col] == -1,
        df[old_col] <= 22.5,
        df[old_col] <= 67.5,
        df[old_col] <= 112.5,
        df[old_col] <= 157.5,
        df[old_col] <= 202.5,
        df[old_col] <= 247.5,
        df[old_col] <= 292.5,
        df[old_col] <= 337.5,
        df[old_col] <= 360]

    values_asp = ["Flat", "North", "Northeast", "East", "Southeast", "South", "Southwest", "West", "Northwest", "North"]
    df[new_col] = np.select(conditions_asp, values_asp)

aspect(df, "Asp", "Aspect")

In [ ]:
#calculate columns for table
df["swe"]= df["SD"]*multiplier
df["swe_mm"]=df["swe"]*1000
df["swe_lm"]=(df["SD"]*l_ci)*1000
df["swe_hm"]=(df["SD"]*h_ci)*1000
df["water_vol"]= df["swe"]*(resolution*resolution)
df["lower_wv"]=df["water_vol"]*lm
df["upper_wv"]=df["water_vol"]*um
df["area"]= resolution

In [ ]:
def pivot_table(value_col, agg_func,name, directory):
    output_name = subbasin_name +"_" + value_col + "_pivot.csv"
    sum = round(pd.crosstab(
      index=df['zbin'], columns=df['Aspect'],
      values=df[value_col], aggfunc=agg_func, margins=True,margins_name="Total").fillna(0),2)
    sum = sum[["Flat","North", "Northeast", "East", "Southeast", "South", "Southwest", "West", "Northwest","Total"]]
    display(sum)
    
    output = os.path.join(directory,output_name)
    sum.to_csv(output)


pivot = pivot_table("water_vol", "sum", name, swe_table_list[wshed])

In [ ]:
def swe_elevation_plots(file_path, file, wshed, sub_basin, x_labels_dates,title,step):
    os.chdir(file_path)
    #set graph parameters
    sns.set_style("darkgrid")
    fig, ax = plt.subplots(figsize=(10,6))
    fig.set_tight_layout(True)
    color_list = sns.color_palette()[:5]
    
    #read in csv
    df = pd.read_csv(file)
    
    #variables per each phase
    p01 = df[df["Phase"]==1]
    p02 = df[df["Phase"]==2]
    p03 = df[df["Phase"]==3]
    p04 = df[df["Phase"]==4]
    p05 = df[df["Phase"]==5]
    
    p1 = ax.plot(p01["zbin"], p01["swe_mm"], "-o", color=color_list[0])
    p2 = ax.plot(p02["zbin"], p02["swe_mm"], "-o", color=color_list[1])
    p3 = ax.plot(p03["zbin"], p03["swe_mm"], "-o", color=color_list[2])
    p4 = ax.plot(p04["zbin"], p04["swe_mm"], "-o", color=color_list[3])
    p5 = ax.plot(p05["zbin"], p05["swe_mm"], "-o", color=color_list[4])
    
    ax.fill_between(p01["zbin"], p01["swe_lm"],p01["swe_hm"], alpha=0.2, color = color_list[0])
    ax.fill_between(p02["zbin"], p02["swe_lm"],p02["swe_hm"], alpha=0.2, color = color_list[1])
    ax.fill_between(p03["zbin"], p03["swe_lm"],p03["swe_hm"], alpha=0.2, color = color_list[2])
    ax.fill_between(p04["zbin"], p04["swe_lm"],p04["swe_hm"], alpha=0.2, color = color_list[3])
    ax.fill_between(p05["zbin"], p05["swe_lm"],p05["swe_hm"], alpha=0.2, color = color_list[4])
    
    ax.set_xlabel('Elevation [m, ellipsoid]')
    tick_spacing = 100
    ax.xaxis.set_major_locator(ticker.MultipleLocator(tick_spacing))
    
    max_swe = (round(df["swe_hm"].max(),-2))+step

    ax.yaxis.set_ticks(np.arange(0, max_swe, step))
    ax.set_ylabel('SWE [mm]')
    y_bottom = 0 - (step/6)
    ax.set_ylim(bottom=y_bottom)
     
    ax.set_xlim(left=p01["zbin"].min()-20, right=(p01["zbin"].max()+20))
    ax.legend([p1,p2,p3,p4,p5], labels=(x_labels_dates), loc="upper left", fontsize = "medium")
    ax.set_title(title, fontsize = 20)
    #save figure, display plot
    plt.savefig("SWE_" + file + ".png", dpi=300)
    plt.show()
    
#call functions    
Van_title = "SWE by Elevation \n Seymour watershed"
Van = swe_elevation_plots(r"H:\Metro-Van\2021\Analysis\swe_tables\zbin","zbin_Seymour_all.csv", "Metro-Van", "Seymour", Metro_dates,Van_title,200)